In [1]:
mv = 0.001
ms = 0.001
mmo = 0.001
na = 1E-9
megohm = 1E6

# typical neuron parameters
threshold = -50 * mv     # spike threshold [mV]
reset_voltage = -65 * mv  # reset potential [mV]
tau_m = 8. * ms     # membrane time constant [ms]
membrane_resistance = 10 * megohm    # membrane resistance [MOhm]
initial_potential = -75 * mv   # initial potential [mV]
resting_potential = -70 * mv      # resting potential [mV]
refactory_period = 3. * ms      # refractory time (ms)
I = 1.0 * na # base injection current [mA]

# simulation parameters
dt = 0.1 * ms  # Simulation time step [ms]
R_DUR = int(refactory_period / dt)

In [2]:
import numpy as np


def initial_state(count):
    potentials = np.full(count, initial_potential)
    ts = np.zeros(count)
    injected_currents = na * (np.array(range(count)) + 1)
    return injected_currents, potentials, ts


def step(v, tr, injected_current):
    rv = np.full_like(v, reset_voltage)
    dv = ((resting_potential - v) + (injected_current * membrane_resistance)) * (dt / tau_m)
    spikes = v > threshold
    next_v = np.where(spikes, rv, v + dv)
    refactory = tr > 0
    next_v = np.where(refactory, rv, next_v)
    next_tr = np.where(refactory, tr - 1, tr)
    R_DUR = int(refactory_period / dt)
    next_tr = np.where(spikes, R_DUR, next_tr)
    return next_v, next_tr, spikes

In [3]:
COUNT = 1000000

injected_currents, potentials, ts = initial_state(COUNT)

%timeit -n 10 -r 10 step(potentials,ts, injected_currents)

13.7 ms ± 351 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


So using `numpy` *a single step for 1,000,000 neurons* takes 13.7 ms!